In [ ]:
# === RESUME EXTRACTION - CHỈ TRÍCH THỰC THỂ ===
!pip install -q langchain-groq pyvis pypdf networkx pydantic python-dotenv

import os
import re
import time
import pickle
import unicodedata
import networkx as nx
from typing import List
from dotenv import load_dotenv

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from pyvis.network import Network
from pydantic import BaseModel, Field

print("✅ Import OK")

In [ ]:
# === CẤU HÌNH ===
load_dotenv()

GROQ_API_KEYS = [
    os.getenv("GROQ_API_KEY_1"),
    os.getenv("GROQ_API_KEY_2"),
    os.getenv("GROQ_API_KEY_3"),
    os.getenv("GROQ_API_KEY_4"),
    os.getenv("GROQ_API_KEY_5"),
    os.getenv("GROQ_API_KEY_6"),
    os.getenv("GROQ_API_KEY_7"),
    os.getenv("GROQ_API_KEY_8"),
    os.getenv("GROQ_API_KEY_9"),
]
PDF_PATH = os.getenv("PDF_PATH")

# Normalize
MEDICAL_ABBREVIATIONS = {
    "btm": "bệnh thận mạn", "tha": "tăng huyết áp", "đtđ": "đái tháo đường",
    "gfr": "độ lọc cầu thận", "ckd": "bệnh thận mạn",
}

MEDICAL_SYNONYMS = {
    "bệnh thận mạn": ["bệnh thận mãn", "suy thận mạn", "ckd"],
    "đái tháo đường": ["tiểu đường", "đtđ", "diabetes"],
}

def normalize_medical_text(text: str) -> str:
    if not text: return "Unknown"
    text = unicodedata.normalize("NFC", text).strip().lower()
    text = re.sub(r'\s+', ' ', text)
    words = [MEDICAL_ABBREVIATIONS.get(re.sub(r'[^\w]', '', w), w) for w in text.split()]
    text = ' '.join(words)
    for canonical, variants in MEDICAL_SYNONYMS.items():
        for variant in variants:
            text = text.replace(variant, canonical)
    return re.sub(r'\s+', ' ', text).strip().title()

class APIKeyManager:
    def __init__(self, api_keys: List[str]):
        self.api_keys = [k for k in api_keys if k]
        self.current_index = 0
        self.failed_keys = set()
    def get_current_key(self) -> str:
        return self.api_keys[self.current_index]
    def rotate_key(self) -> bool:
        self.failed_keys.add(self.current_index)
        for i in range(len(self.api_keys)):
            next_idx = (self.current_index + 1 + i) % len(self.api_keys)
            if next_idx not in self.failed_keys:
                self.current_index = next_idx
                print(f"🔄 Key #{next_idx + 1}")
                return True
        return False
    def reset_failed(self):
        self.failed_keys.clear()

api_manager = APIKeyManager(GROQ_API_KEYS)
print(f"✅ {len(api_manager.api_keys)} API keys")

In [ ]:
# === SCHEMA ===
class MedicalEntity(BaseModel):
    name: str
    type: str
    description: str = ""
    relevance_score: int = Field(default=5, ge=1, le=10)

class MedicalRelation(BaseModel):
    source_name: str
    target_name: str
    relation: str
    confidence_score: int = Field(default=5, ge=1, le=10)
    evidence: str = ""

class AMGExtractionResult(BaseModel):
    entities: List[MedicalEntity] = Field(default_factory=list)
    relations: List[MedicalRelation] = Field(default_factory=list)

print("✅ Schema OK")

In [ ]:
# === LOAD CHECKPOINT ===
graph_path = "./amg_data/graph_full.pkl"

if os.path.exists(graph_path):
    with open(graph_path, "rb") as f:
        G = pickle.load(f)
    print(f"✅ Loaded: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
else:
    print("⚠️ Không tìm thấy checkpoint, tạo graph mới")
    G = nx.DiGraph()

In [ ]:
# === OPTIMIZED EXTRACTOR ===
class OptimizedAMGExtractor:
    def __init__(self, api_manager: APIKeyManager):
        self.api_manager = api_manager
        self.extraction_prompt = ChatPromptTemplate.from_messages([
            ("system", """Bạn là chuyên gia trích xuất thực thể y tế.

## ENTITY TYPES:
DISEASE, DRUG, SYMPTOM, TEST, ANATOMY, TREATMENT, PROCEDURE, RISK_FACTOR, LAB_VALUE

## SCORING:
- 9-10: Thực thể CHÍNH (đề cập nhiều lần)
- 7-8: QUAN TRỌNG
- 5-6: PHỤ
- 3-4: LIÊN QUAN nhẹ

## EXAMPLES:
Input: "Bệnh thận mạn giai đoạn 5 cần lọc máu. Erythropoietin khi Hb < 10g/dL."
Output:
- Bệnh thận mạn giai đoạn 5 (DISEASE, 10)
- Lọc máu (TREATMENT, 9)
- Erythropoietin (DRUG, 8)

## RULES:
1. GIỮ NGUYÊN tiếng Việt
2. KHÔNG extract: số trang, quyết định, văn bản

Trích xuất:"""),
            ("human", "{text}")
        ])
        self._init_llm()
    
    def _init_llm(self):
        self.llm = ChatGroq(temperature=0.1, model="llama-3.3-70b-versatile", 
                           api_key=self.api_manager.get_current_key())
        self.chain = self.extraction_prompt | self.llm.with_structured_output(AMGExtractionResult)
    
    def extract(self, text: str, max_retries=3):
        for attempt in range(max_retries):
            try:
                result = self.chain.invoke({"text": text})
                if result:
                    # Filter
                    valid = [e for e in result.entities 
                            if len(e.name) >= 2 and 
                            not any(x in e.name.lower() for x in ['quyết định', 'văn bản', 'trang'])]
                    result.entities = valid
                return result if result else AMGExtractionResult()
            except Exception as e:
                if "rate" in str(e).lower() or "limit" in str(e).lower():
                    if self.api_manager.rotate_key():
                        self._init_llm()
                        time.sleep(2)
                    else:
                        print("⏳ Chờ 5 phút...")
                        time.sleep(300)
                        self.api_manager.reset_failed()
                        self._init_llm()
                else:
                    time.sleep(1)
        return AMGExtractionResult()

print("✅ Extractor OK")

In [ ]:
# === HELPER: ADD TO GRAPH ===
def add_entity(G, entity: MedicalEntity, page_num: int):
    norm_name = normalize_medical_text(entity.name)
    confidence = min(1.0, entity.relevance_score / 10.0)
    
    if not G.has_node(norm_name):
        G.add_node(norm_name, label=entity.name, type=entity.type.upper(),
                  description=entity.description, confidence=confidence,
                  relevance_score=entity.relevance_score, pages=[page_num])
    else:
        old_conf = G.nodes[norm_name].get('confidence', 0)
        if confidence > old_conf:
            G.nodes[norm_name]['confidence'] = confidence
            G.nodes[norm_name]['description'] = entity.description
        if page_num not in G.nodes[norm_name]['pages']:
            G.nodes[norm_name]['pages'].append(page_num)

def add_relation(G, rel: MedicalRelation, page_num: int):
    src = normalize_medical_text(rel.source_name)
    tgt = normalize_medical_text(rel.target_name)
    if G.has_node(src) and G.has_node(tgt):
        G.add_edge(src, tgt, relation=rel.relation.upper(),
                  confidence=min(1.0, rel.confidence_score/10), 
                  evidence=rel.evidence, page=page_num)

print("✅ Helpers OK")

In [ ]:
# === EXTRACT TỪ CHUNK 121 ===
START_CHUNK = 120
START_PAGE = 17

print(f"🚀 RESUME TỪ CHUNK {START_CHUNK + 1}\n")

# Load PDF
loader = PyPDFLoader(PDF_PATH)
pages = loader.load()[START_PAGE-1:]
print(f"✂️ Cắt {START_PAGE-1} trang, còn {len(pages)} trang")

# Split
splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(pages)
print(f"📦 Tổng: {len(chunks)} chunks")
print(f"▶️  Xử lý: chunk {START_CHUNK + 1} → {len(chunks)}\n")

# Extract
extractor = OptimizedAMGExtractor(api_manager)
chunks_to_process = chunks[START_CHUNK:]

print("🧠 ĐANG TRÍCH XUẤT (Optimized Prompts)...\n")

for i, chunk in enumerate(chunks_to_process, start=START_CHUNK+1):
    page_num = chunk.metadata.get("page", 0) + 1
    result = extractor.extract(chunk.page_content)
    
    if result and len(result.entities) > 0:
        avg_rel = sum(e.relevance_score for e in result.entities) / len(result.entities)
        print(f"   [{i}/{len(chunks)}] +{len(result.entities)} entities (avg: {avg_rel:.1f})")
        
        for ent in result.entities:
            add_entity(G, ent, page_num)
        
        for rel in result.relations:
            add_relation(G, rel, page_num)
    else:
        print(f"   [{i}/{len(chunks)}] --")
    
    # Save mỗi 20 chunks
    if i % 20 == 0:
        with open("./amg_data/graph_full.pkl", "wb") as f:
            pickle.dump(G, f)
        print(f"   💾 Saved: {G.number_of_nodes()} nodes\n")
    
    time.sleep(1)

print(f"\n✅ XONG!")
print(f"Final: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")

In [ ]:
# === SAVE ===
with open("./amg_data/graph_full.pkl", "wb") as f:
    pickle.dump(G, f)

print(f"💾 Đã lưu graph_full.pkl")
print(f"   {G.number_of_nodes()} nodes | {G.number_of_edges()} edges")

In [ ]:
# === VISUALIZE ===
net = Network(height="850px", width="100%", directed=True, notebook=False)
net.force_atlas_2based(gravity=-50, spring_length=200)

color_map = {
    'DISEASE': '#ff6b6b', 'DRUG': '#4ecdc4', 'SYMPTOM': '#ffe66d',
    'TEST': '#95e1d3', 'TREATMENT': '#aa96da'
}

for node, data in G.nodes(data=True):
    color = color_map.get(data.get('type', 'OTHER'), '#97C2FC')
    conf = data.get('confidence', 0.5)
    size = 15 + conf * 25
    net.add_node(node, label=data.get('label'), color=color, size=size,
                title=f"{data.get('label')}<br>Conf: {conf:.2f}")

for u, v, data in G.edges(data=True):
    width = 1 + data.get('confidence', 0.5) * 3
    net.add_edge(u, v, label=data.get('relation'), width=width)

net.show_buttons(filter_=['physics'])
net.write_html("Medical_Graph_Final.html")

print("✅ Medical_Graph_Final.html")
print("🎉 HOÀN TẤT!")